<a href="https://colab.research.google.com/github/hsandaver/hsandaver/blob/main/UVExposureSimulatorv1_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Cultural Heritage UV Exposure and Color Analysis
### Simulating UV Light Effects on Images and Analyzing Color Changes

# Import Libraries and Upload LAB Dataset

In [ ]:
!pip install colormath
# Import necessary libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.spatial import KDTree
from PIL import Image
from colormath.color_objects import sRGBColor, LabColor
from colormath.color_conversions import convert_color
import matplotlib.patches as patches
from google.colab import files

# Upload the CSV file containing LAB color dataset
print("Please upload your LAB color dataset CSV file.")
uploaded_files = files.upload()
csv_filename = next(iter(uploaded_files))

# Load the dataset
dataset = pd.read_csv(csv_filename)

# Clean the dataset by removing rows with NaN or infinite values
dataset = dataset.replace([np.inf, -np.inf], np.nan).dropna(subset=['L', 'A', 'B'])

# Extract LAB values
lab_values = dataset[['L', 'A', 'B']].values

# Create the KDTree with the cleaned data
lab_tree = KDTree(lab_values)

# Load and Display Original Image

In [ ]:
# Step 2: Load and display the original image
print("Please upload the image file you want to analyze.")
uploaded_images = files.upload()
image_filename = next(iter(uploaded_images))
image = Image.open(image_filename).convert('RGB')
plt.figure(figsize=(6, 6))
plt.imshow(image)
plt.axis('off')
plt.title('Original Image')
plt.show()

# UV Exposure Simulation Function

In [ ]:
# UV Exposure Simulation Function
def simulate_uv_exposure(image, exposure_years=1, exposure_factor=36.5, max_exposure_hours=144):
    """
    Simulate UV exposure by adjusting LAB color values based on the number of exposure years.
    Each museum year is approximately equivalent to 36.5 hours of accelerated UV exposure.
    :param image: Input image (PIL Image format)
    :param exposure_years: Number of years of simulated UV exposure
    :param exposure_factor: Accelerated UV exposure equivalent to 1 museum year (default 36.5 hours)
    :param max_exposure_hours: Maximum accelerated UV exposure time (default 144 hours)
    """
    # Convert image to numpy array
    image_array = np.array(image)

    # Convert RGB to LAB
    lab_image = np.zeros_like(image_array, dtype=float)

    for i in range(image_array.shape[0]):
        for j in range(image_array.shape[1]):
            # Convert each pixel from RGB to LAB
            rgb_pixel = sRGBColor(*(image_array[i, j] / 255.0))
            lab_pixel = convert_color(rgb_pixel, LabColor)

            # Simulate UV exposure by modifying the b* value
            current_exposure_hours = exposure_years * exposure_factor
            exposure_ratio = min(current_exposure_hours / max_exposure_hours, 1.0)

            # Increase yellowing effect on the b* axis
            lab_pixel.lab_b += exposure_ratio * (lab_pixel.lab_b * 0.2)  # Increase b* by 20%

            # Store the LAB values back
            lab_image[i, j] = [lab_pixel.lab_l, lab_pixel.lab_a, lab_pixel.lab_b]

    # Convert LAB back to RGB
    uv_exposed_image = np.zeros_like(image_array, dtype=float)

    for i in range(image_array.shape[0]):
        for j in range(image_array.shape[1]):
            lab_pixel = LabColor(*lab_image[i, j])
            rgb_pixel = convert_color(lab_pixel, sRGBColor)

            # Convert back to 8-bit RGB values
            uv_exposed_image[i, j] = [
                max(0, min(255, rgb_pixel.rgb_r * 255)),
                max(0, min(255, rgb_pixel.rgb_g * 255)),
                max(0, min(255, rgb_pixel.rgb_b * 255))
            ]

    # Convert the result back to an image
    uv_exposed_image = uv_exposed_image.astype(np.uint8)
    return Image.fromarray(uv_exposed_image)

# Simulate UV exposure for 5 years
simulated_uv_image = simulate_uv_exposure(image, exposure_years=5)

# Display the simulated image
plt.imshow(simulated_uv_image)
plt.title('Simulated UV Exposure (5 years)')
plt.axis('off')
plt.show()

# Extract and Compare Colors with LAB Dataset

In [ ]:
# Convert image to numpy array and calculate average color
image_array = np.array(simulated_uv_image)
average_color = image_array.mean(axis=(0, 1)) / 255.0

# Display average color patch
average_color_patch = np.ones((100, 100, 3)) * average_color
plt.figure(figsize=(2, 2))
plt.imshow(average_color_patch)
plt.axis('off')
plt.title('Average Color Extracted')
plt.show()

# Convert the average color to LAB
rgb_color = sRGBColor(*average_color)
lab_color = convert_color(rgb_color, LabColor)
L, A, B = lab_color.lab_l, lab_color.lab_a, lab_color.lab_b

# Find closest colors in the dataset
distances, indices = lab_tree.query([L, A, B], k=5)
closest_colors = dataset.iloc[indices]

# Display closest colors and their names
fig, axs = plt.subplots(1, 5, figsize=(15, 3))
for i, (index, dist) in enumerate(zip(indices, distances)):
    closest_color = dataset.iloc[index]
    lab_color_closest = LabColor(closest_color['L'], closest_color['A'], closest_color['B'])
    rgb_color_closest = convert_color(lab_color_closest, sRGBColor)

    rgb_values_closest = [max(0, min(1, val)) for val in [rgb_color_closest.rgb_r, rgb_color_closest.rgb_g, rgb_color_closest.rgb_b]]
    color_patch = np.ones((100, 100, 3)) * rgb_values_closest
    axs[i].imshow(color_patch)
    axs[i].axis('off')
    axs[i].set_title(f"{closest_color['Color Name']}\nDist: {dist:.2f}")

plt.show()

# Pixel-by-Pixel Color Difference Mapping

In [ ]:
### Pixel-by-Pixel Color Difference Mapping ###
def color_difference_map(image1, image2, threshold=2):
    # Resize image1 to match the size of image2
    image1 = image1.resize(image2.size)

    image1_array = np.array(image1)
    image2_array = np.array(image2)

    diff_image = np.zeros_like(image1_array[..., 0], dtype=np.float32)  # ∆E map

    for i in range(image1_array.shape[0]):
        for j in range(image1_array.shape[1]):
            # Convert RGB values to LAB color space for both images
            rgb1 = sRGBColor(*(image1_array[i, j] / 255.0))
            lab1 = convert_color(rgb1, LabColor)

            rgb2 = sRGBColor(*(image2_array[i, j] / 255.0))
            lab2 = convert_color(rgb2, LabColor)

            # Calculate the color difference (∆E)
            delta_e = np.sqrt((lab1.lab_l - lab2.lab_l)**2 +
                              (lab1.lab_a - lab2.lab_a)**2 +
                              (lab1.lab_b - lab2.lab_b)**2)

            diff_image[i, j] = delta_e  # Store the ∆E value

    # Detectable fading: Count pixels with ∆E above the threshold
    detectable_fading = np.sum(diff_image > threshold)
    total_pixels = diff_image.size
    percentage_detectable = (detectable_fading / total_pixels) * 100

    print(f"Percentage of image with detectable fading (∆E > {threshold}): {percentage_detectable:.2f}%")

    return diff_image

# Step 3: Compare pixel-by-pixel color difference between two images (original vs faded)
print("Please upload the faded image for comparison.")
uploaded_faded_images = files.upload()
faded_image_filename = next(iter(uploaded_faded_images))
faded_image = Image.open(faded_image_filename).convert('RGB')

# Run color difference map with a threshold for detectable differences
color_diff = color_difference_map(simulated_uv_image, faded_image, threshold=2)

# Display the color difference map
plt.imshow(color_diff, cmap='hot')
plt.colorbar()
plt.title("Pixel-by-Pixel Color Difference Map (∆E)")
plt.show()

In [ ]:
def simulate_visible_light_exposure(image, exposure_years=1, exposure_factor=36.5, max_exposure_hours=144):
    """
    Simulate visible light exposure by adjusting LAB color values based on the number of exposure years.
    Each museum year is approximately equivalent to 36.5 hours of visible light exposure.
    :param image: Input image (PIL Image format)
    :param exposure_years: Number of years of simulated light exposure
    :param exposure_factor: Accelerated light exposure equivalent to 1 museum year (default 36.5 hours)
    :param max_exposure_hours: Maximum accelerated exposure time (default 144 hours)
    """
    # Convert image to numpy array
    image_array = np.array(image)

    # Convert RGB to LAB
    lab_image = np.zeros_like(image_array, dtype=float)

    for i in range(image_array.shape[0]):
        for j in range(image_array.shape[1]):
            # Convert each pixel from RGB to LAB
            rgb_pixel = sRGBColor(*(image_array[i, j] / 255.0))
            lab_pixel = convert_color(rgb_pixel, LabColor)

            # Simulate visible light exposure by modifying the L*, a*, and b* values
            current_exposure_hours = exposure_years * exposure_factor
            exposure_ratio = min(current_exposure_hours / max_exposure_hours, 1.0)

            # Decrease lightness (L*) slightly to simulate slight fading of pigments
            lab_pixel.lab_l -= exposure_ratio * (lab_pixel.lab_l * 0.05)  # 5% lightness reduction

            # Subtle shift towards neutral for a* (red-green axis) and b* (blue-yellow axis)
            lab_pixel.lab_a -= exposure_ratio * (lab_pixel.lab_a * 0.03)  # 3% reduction in a*
            lab_pixel.lab_b -= exposure_ratio * (lab_pixel.lab_b * 0.03)  # 3% reduction in b*

            # Store the modified LAB values back
            lab_image[i, j] = [lab_pixel.lab_l, lab_pixel.lab_a, lab_pixel.lab_b]

    # Convert LAB back to RGB
    visible_light_exposed_image = np.zeros_like(image_array, dtype=float)

    for i in range(image_array.shape[0]):
        for j in range(image_array.shape[1]):
            lab_pixel = LabColor(*lab_image[i, j])
            rgb_pixel = convert_color(lab_pixel, sRGBColor)

            # Convert back to 8-bit RGB values
            visible_light_exposed_image[i, j] = [
                max(0, min(255, rgb_pixel.rgb_r * 255)),
                max(0, min(255, rgb_pixel.rgb_g * 255)),
                max(0, min(255, rgb_pixel.rgb_b * 255))
            ]

    # Convert the result back to an image
    visible_light_exposed_image = visible_light_exposed_image.astype(np.uint8)
    return Image.fromarray(visible_light_exposed_image)

In [ ]:
# Simulate visible light exposure
def simulate_visible_light_exposure(image, exposure_years=1, exposure_factor=36.5, max_exposure_hours=144):
    """
    Simulate visible light exposure by adjusting LAB color values based on the number of exposure years.
    Each museum year is approximately equivalent to 36.5 hours of visible light exposure.
    :param image: Input image (PIL Image format)
    :param exposure_years: Number of years of simulated light exposure
    :param exposure_factor: Accelerated light exposure equivalent to 1 museum year (default 36.5 hours)
    :param max_exposure_hours: Maximum accelerated exposure time (default 144 hours)
    """
    # Convert image to numpy array
    image_array = np.array(image)

    # Convert RGB to LAB
    lab_image = np.zeros_like(image_array, dtype=float)

    for i in range(image_array.shape[0]):
        for j in range(image_array.shape[1]):
            # Convert each pixel from RGB to LAB
            rgb_pixel = sRGBColor(*(image_array[i, j] / 255.0))
            lab_pixel = convert_color(rgb_pixel, LabColor)

            # Simulate visible light exposure by modifying the L*, a*, and b* values
            current_exposure_hours = exposure_years * exposure_factor
            exposure_ratio = min(current_exposure_hours / max_exposure_hours, 1.0)

            # Decrease lightness (L*) slightly to simulate slight fading of pigments
            lab_pixel.lab_l -= exposure_ratio * (lab_pixel.lab_l * 0.05)  # 5% lightness reduction

            # Subtle shift towards neutral for a* (red-green axis) and b* (blue-yellow axis)
            lab_pixel.lab_a -= exposure_ratio * (lab_pixel.lab_a * 0.03)  # 3% reduction in a*
            lab_pixel.lab_b -= exposure_ratio * (lab_pixel.lab_b * 0.03)  # 3% reduction in b*

            # Store the modified LAB values back
            lab_image[i, j] = [lab_pixel.lab_l, lab_pixel.lab_a, lab_pixel.lab_b]

    # Convert LAB back to RGB
    visible_light_exposed_image = np.zeros_like(image_array, dtype=float)

    for i in range(image_array.shape[0]):
        for j in range(image_array.shape[1]):
            lab_pixel = LabColor(*lab_image[i, j])
            rgb_pixel = convert_color(lab_pixel, sRGBColor)

            # Convert back to 8-bit RGB values
            visible_light_exposed_image[i, j] = [
                max(0, min(255, rgb_pixel.rgb_r * 255)),
                max(0, min(255, rgb_pixel.rgb_g * 255)),
                max(0, min(255, rgb_pixel.rgb_b * 255))
            ]

    # Convert the result back to an image
    visible_light_exposed_image = visible_light_exposed_image.astype(np.uint8)
    return Image.fromarray(visible_light_exposed_image)


# Pixel-by-pixel color difference map and percentage of detectable fading
def color_difference_map(image1, image2, threshold=2):
    # Resize image1 to match the size of image2
    image1 = image1.resize(image2.size)

    image1_array = np.array(image1)
    image2_array = np.array(image2)

    diff_image = np.zeros_like(image1_array[..., 0], dtype=np.float32)  # ∆E map

    for i in range(image1_array.shape[0]):
        for j in range(image1_array.shape[1]):
            # Convert RGB values to LAB color space for both images
            rgb1 = sRGBColor(*(image1_array[i, j] / 255.0))
            lab1 = convert_color(rgb1, LabColor)

            rgb2 = sRGBColor(*(image2_array[i, j] / 255.0))
            lab2 = convert_color(rgb2, LabColor)

            # Calculate the color difference (∆E)
            delta_e = np.sqrt((lab1.lab_l - lab2.lab_l)**2 +
                              (lab1.lab_a - lab2.lab_a)**2 +
                              (lab1.lab_b - lab2.lab_b)**2)

            diff_image[i, j] = delta_e  # Store the ∆E value

    # Detectable fading: Count pixels with ∆E above the threshold
    detectable_fading = np.sum(diff_image > threshold)
    total_pixels = diff_image.size
    percentage_detectable = (detectable_fading / total_pixels) * 100

    print(f"Percentage of image with detectable fading (∆E > {threshold}): {percentage_detectable:.2f}%")

    return diff_image

# Load the original image and simulate visible light exposure
print("Please upload the original image.")
uploaded_images = files.upload()
image_filename = next(iter(uploaded_images))
original_image = Image.open(image_filename).convert('RGB')

# Simulate visible light exposure for 5 years
simulated_visible_light_image = simulate_visible_light_exposure(original_image, exposure_years=5)

# Display the simulated visible light exposed image
plt.imshow(simulated_visible_light_image)
plt.title('Simulated Visible Light Exposure (5 years)')
plt.axis('off')
plt.show()

# Step 3: Compare pixel-by-pixel color difference between original and visible light exposed image
color_diff = color_difference_map(original_image, simulated_visible_light_image, threshold=2)

# Display the color difference map with a shorter title
plt.imshow(color_diff, cmap='hot')
plt.colorbar()

# Adjust the title placement and spacing
plt.title("Pixel-by-Pixel ∆E Map (Visible Light)", pad=20)  # Shortened title and padding
plt.tight_layout()  # Ensure layout is clean
plt.show()

# Histogram Analysis for RGB and LAB

In [ ]:
### Updated Feature 3: Histogram Analysis (RGB and LAB) ###
def plot_histograms(image1, image2):
    """
    Plot RGB and LAB histograms for two images.
    """
    # Convert images to numpy arrays
    image1_array = np.array(image1)
    image2_array = np.array(image2)

    # Check that images have been loaded and are the same size
    print("Image1 shape:", image1_array.shape)
    print("Image2 shape:", image2_array.shape)

    # Flatten the RGB values for both images
    image1_flat = image1_array.reshape(-1, 3)
    image2_flat = image2_array.reshape(-1, 3)

    fig, axs = plt.subplots(2, 2, figsize=(12, 8))

    # RGB Histogram for Red Channel
    axs[0, 0].hist(image1_flat[:, 0], bins=256, color='r', alpha=0.5, label='Red (Original)')
    axs[0, 0].hist(image2_flat[:, 0], bins=256, color='b', alpha=0.5, label='Red (Faded)')
    axs[0, 0].set_title('Red Channel Histogram (Original vs Faded)')
    axs[0, 0].legend()

    # RGB Histogram for Green Channel
    axs[0, 1].hist(image1_flat[:, 1], bins=256, color='g', alpha=0.5, label='Green (Original)')
    axs[0, 1].hist(image2_flat[:, 1], bins=256, color='y', alpha=0.5, label='Green (Faded)')
    axs[0, 1].set_title('Green Channel Histogram (Original vs Faded)')
    axs[0, 1].legend()

    # RGB Histogram for Blue Channel
    axs[1, 0].hist(image1_flat[:, 2], bins=256, color='b', alpha=0.5, label='Blue (Original)')
    axs[1, 0].hist(image2_flat[:, 2], bins=256, color='cyan', alpha=0.5, label='Blue (Faded)')
    axs[1, 0].set_title('Blue Channel Histogram (Original vs Faded)')
    axs[1, 0].legend()

    # Convert RGB to LAB
    image1_lab = np.array([convert_color(sRGBColor(*pixel/255.0), LabColor).get_value_tuple() for pixel in image1_flat])
    image2_lab = np.array([convert_color(sRGBColor(*pixel/255.0), LabColor).get_value_tuple() for pixel in image2_flat])

    # LAB Histogram for L* (Lightness)
    axs[1, 1].hist(image1_lab[:, 0], bins=256, color='magenta', alpha=0.5, label='L* (Original)')
    axs[1, 1].hist(image2_lab[:, 0], bins=256, color='orange', alpha=0.5, label='L* (Faded)')
    axs[1, 1].set_title('LAB L* Histogram (Original vs Faded)')
    axs[1, 1].legend()

    # Adjust the layout to avoid overlap
    plt.tight_layout()

    # Show the plot
    plt.show()

# Call the updated function to plot histograms
plot_histograms(simulated_uv_image, faded_image)